In [64]:
import numpy as np
import math

t = 0  # number of this round
T = 8000  # numbers of rounds
l = 30  # ||a|| <= L
d = 3 #dimension of arm
m = 1 #||theta|| < m
theta_stimato = [0.0, 0.0, 0.0] #np.array([])  theta
A = [] #collector of arm
num_arms = 0 #number of arms
num_theta = 0 #num theta
Th = [] #collector of theta
W = [] #distribution of probability to play arms
N = [] #number of times an arm is played
Y = [] #collection of reward
H = [] #collection of hedge losses
L = [] #collection of losses
Mixability_Gap = [] #collection of all mixability gap at each t
AI = [] #number of the arm played at time I
nu = 0.01 #regularization paramether
eta = 0.1 #learning paramether
theta_vero = 0
curr_answ = 0

def normalize(v):
    sum = np.sum(v)
    if sum > 0:
        return v / sum
    else:
        return np.array([0.33]*len(v))
    
class theta():
    vector = np.array([])
    best_arm = -1
    id

    def __init__(self, param):
        self.vector = param
        global Th, num_theta
        self.id = num_theta
        Th.append(self)
        num_theta +=1
        #i = 0
        #best = -5
        #while i < num_arms:
            #tmp = np.dot( A[i].arm_vector, np.transpose(self.vector))
            #if best < tmp:
                #best = tmp
                #self.best_arm = i

class arm():
    arm_vector = np.array([])
    id = 0
    best_theta = -1

    def __init__(self, vector):
        self.arm_vector = vector
        global num_arms, A
        A.append(self)
        W.append(0)
        N.append(0)
        L.append(0)
        self.id = num_arms
        num_arms +=1
        max = -5
        for i in range (0, num_theta):
            tmp = np.matmul(self.arm_vector, Th[i].vector)
            if tmp > max:
                self.best_theta = i
                max = tmp
                            

    def pull_arm(self):
        y = np.dot(self.arm_vector, np.transpose(Th[theta_vero].vector)) + np.random.normal(0, 1)
        Y.append(y)
        N[self.id] += 1
        AI.append(self.id)
        return y
        
    def norm(self):
        return np.linalg.norm(self.arm_vector)

def design_matrix(w):
    V = []
    i = 0
    while i < num_arms:
        b = [A[i].arm_vector]
        a = np.swapaxes(b, 0, 1)
        ris = np.matmul(a, b) * w[i]
        if (i == 0):
            V = ris
        else:
            V += ris
        i+=1
    return V

def norm_2_mat(x, V):
    b = [x]
    a = np.swapaxes(b, 0, 1)
    ris = np.sqrt(np.matmul(np.matmul(b,V),a))
    return ris[0][0]

def arm_to_play():
    diff = np.array(N) - np.array(W)
    i = 0
    arm = 0
    min = +5000
    while i < num_arms:
        if diff[i] < min:
            min = diff[i]
            arm = i
        i += 1
    return arm

def stima_theta():
    V = design_matrix(N)
    I = np.identity(d)
    p = np.linalg.inv(V + nu*I)
    sum = 0
    for i in range (0, t):
        sum += Y[i] * np.array(A[AI[i]].arm_vector)
    res = np.matmul(p, sum)
    global theta_stimato
    theta_stimato = res
    return res

def best_answer():
    max = -100
    best = -1
    for i in range (0, num_arms):
        curr = np.matmul(A[i].arm_vector, theta_stimato)
        if curr > max:
            max = curr
            best = i
    global curr_answ
    curr_answ = best
    return best

def best_arm():
    max = -100
    best = -1
    for i in range (0, num_arms):
        curr = np.matmul(A[i].arm_vector, Th[theta_vero].vector)
        if curr > max:
            max = curr
            best = i
    global curr_answ
    curr_answ = best
    return best


def beta(time, delta):
    n = np.abs(np.random.normal(0, 1))
    res = math.sqrt(math.log10(1/delta) + d/2*math.log10(1 + (time*(l**2))/(n*d))) + math.sqrt(n/2)*m
    return res

def best_response():
    min = 1000000
    theta_adv = -11
    for i in range (0, num_theta):
        if not i == A[curr_answ].best_theta:
            curr = (norm_2_mat(np.array(theta_stimato) - np.array(Th[i].vector), design_matrix(W)))**2
            if curr < min:
                min = curr
                theta_adv = i
    return theta_adv

def calcola_U (id_arm):
    xi = [0]*d
    q2 = 4*(l**2)*(m**2)
    q1 = 0
    p1 = np.matmul((np.array(theta_stimato) - np.array(Th[best_response()].vector)), np.array(A[id_arm].arm_vector))
    p2 = math.sqrt(2*beta(T, 1/T**3))*norm_2_mat(np.array(A[id_arm].arm_vector), design_matrix(normalize(N)) + np.random.normal(0, 1)*np.identity(d))
    r1 = (p1 + p2)**2
    r2 = (p1 - p2)**2
    if r1 > r2:
        q1 = r1
    else:
        q1 = r2
    if q1 > q2:
        return q2
    else:
        return q1

def calcola_loss():
    loss = 0
    arm_play = arm_to_play()
    best_res = best_response ()
    for i in range (0, num_arms):
        loss += W[i]*calcola_U(i)/2
    L[arm_play] += (4*(l**2)*(m**2) - loss)/1000
    return (4*(l**2)*(m**2) - loss)/1000

In [65]:
len([0,1])

2

In [66]:
s = theta([0.1, 0.4, 0.5])
p = theta([0.4, 0.6, 0.2])
q = theta([0.01, 0.1, 0.8])

In [67]:
a = arm([1,2,3])
b = arm([2,5,1])
c = arm([1, 3, 1])
w = [0.34, 0.33, 0.33]
V = design_matrix(w)

In [68]:
np.linalg.norm(c.arm_vector)

3.3166247903554

In [69]:
for i in range (0, num_arms):
    print(A[i].best_theta)

2
1
1


In [70]:
arm_to_play()

0

In [71]:
for i in range (0, num_arms):
    print(np.matmul(A[i].arm_vector, Th[theta_vero].vector))

2.4
2.7
1.8000000000000003


In [60]:
def h():
    return np.dot(W, L)

def mix_loss():
    sum = 0
    for i in range (0, num_arms):
        sum += W[i] * math.exp(-eta * L[i])
    return -1/eta * np.log(sum)

def mixability_gap():
    m_g = h() - mix_loss()
    Mixability_Gap.append(m_g)

def new_eta():
    if t>0:
        sum = 0
        for i in range (0, t):
            sum += Mixability_Gap[i]
        eta = math.log(d)/sum
        
def generate_w():
    sum=0
    for i in range (0, num_arms):
        W[i] =math.exp(-eta * L[i])/num_arms
        sum += math.exp(-eta * L[i])/num_arms
    for i in range (0, num_arms):
        W[i] = W[i]/sum
    return W

In [61]:
a.pull_arm()

3.3133521838989672

In [62]:
t += 1
stima_theta()
best_answer()
curr = arm_to_play()
mixability_gap()
generate_w()
err = calcola_loss()
print(err)
A[curr].pull_arm()

2.55301134644588


C:\Users\edoar\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: RuntimeWarning: divide by zero encountered in log
  


2.40779786067434

In [63]:
t += 1
stima_theta()
best_answer()
curr = arm_to_play()
mixability_gap()
new_eta()
generate_w()
err = calcola_loss()
print(err)
A[curr].pull_arm()

IndexError: list index out of range

In [48]:
N

[1, 1, 1]

In [49]:
normalize(N)

array([0.33333333, 0.33333333, 0.33333333])

In [50]:
for i in range (3000):
    t += 1
    stima_theta()
    best_answer()
    curr = arm_to_play()
    mixability_gap()
    new_eta()
    generate_w()
    err = calcola_loss()
    print(err)
    A[curr].pull_arm()

2.362644265614998
2.383076847813554
2.2506090137800294
2.2681054892389283
2.2996039207928693
2.261228261898128
2.322060229657943
2.310205904455666
2.218664897027812
2.2744573816656697
2.2818727171172792
2.3535267377481692
2.2136028370709773
2.320980055889475
2.2931912727294437
2.3133103069346554
2.281516035179977
2.3310100060079506
2.2842219002316764
2.3388512000571593
2.2476801363797683
2.3607673237347444
2.298214958024976
2.260170321771439
2.3232679609467013
2.303750672952499
2.2342003566602635
2.2909131403307113
2.2739296635712445
2.2557806591680727
2.4014103830894036
2.336212248314475
2.298343221653482
2.276434577782593
2.3102961872046146
2.2709345646580297
2.2750860732346028
2.3430831479712815
2.280319733996411
2.2056730511604474
2.3905618262966692
2.205994183300317
2.212420187550403
2.237537787509532
2.2105469897183814
2.3110319396802845
2.2868462440110173
2.2737589473796915
2.253828855620931
2.314324074950133
2.369002778347106
2.2207332688952732
2.3402764169311636
2.342868719968

2.125334183656912
2.2153519899553897
2.23292798680098
2.2099493154902556
2.2603541880406053
2.1822541910462636
2.208074408027738
2.2304173104815477
2.1691741222684944
2.219621364924211
2.218459127796425
2.233186664054066
2.2087089125434924
2.2843547198358194
2.2733050373385257
2.1909483724749084
2.272708545204312
2.262467821393948
2.173931761135969
2.2443498314214514
2.18285913210335
2.2663504664038934
2.231087811179483
2.181275913278585
2.2155572584997683
2.2242053460790605
2.172604378454386
2.2096733142849647
2.2501134716679316
2.2053261722759623
2.1936023329791925
2.2735064788000456
2.169714698132905
2.1232459983515275
2.2362513665007073
2.1964823527225366
2.2029000615070524
2.217869767792341
2.1920480327568566
2.162882348016534
2.2516253440541707
2.2119698640552263
2.164246088827743
2.2639488744585305
2.229127278921856
2.216566412071853
2.2086024281821355
2.239045181355128
2.237304190596322
2.233051201687588
2.215604406123871
2.1923518690470267
2.177216746087144
2.243263737653434
2

2.14518794607157
2.0792399512193414
2.177564520875704
2.134270274926154
2.101509698349257
2.1643680070481484
2.116221800183345
2.0828414312945736
2.1328723804650007
2.1143123217084514
2.097402717693288
2.1439446201799384
2.095825336837166
2.0313518454124817
2.0868398132025088
2.1149601557086366
2.0979789413778387
2.1439427335308046
2.0888280016825256
2.058162541930526
2.1149268017926577
2.1397390751538166
2.1241660461855134
2.1429089045700067
2.0791377307320342
2.0825964050440393
2.0864786268361795
2.1299709170820234
2.0867934700815858
2.122991623695948
2.127536016818595
2.099691299268527
2.1729068740486213
2.1156490004412674
2.097045168083219
2.1692540069698847
2.124483623632394
2.1118528340292504
2.1494726777963504
2.109179394367947
2.113706348974713
2.161320470006429
2.112611684943225
2.121146073847791
2.1306328755859867
2.124758044744219
2.085608960452508
2.1340278100461108
2.0959731502103223
2.1125905118942443
2.158988326141918
2.1251671169657302
2.0964413260119694
2.1457166394460

2.0520597844763095
2.021885093071034
2.028028931067551
2.061537342950661
2.0565308779634197
2.0341058419864697
2.078083679280736
2.051695938242298
2.0271782848168156
2.0625238432933037
2.0329362792915653
1.996088893378601
2.0827040261054033
2.065863204228512
2.050002266746232
2.08589142593784
2.046964724897684
2.033532047853061
2.0857675577592967
2.0654594282666787
2.039758991752457
2.0761292283542914
2.0347441951919665
2.022728069674649
2.09570091396756
2.0354184918833154
2.062943044289353
2.062398380665685
2.00992352988754
2.03685149416378
2.062316924384334
2.072179837537024
2.018108954041128
2.0587878843601
2.0194762865651006
2.0349232510139275
2.070201070277259
2.041081039620564
2.0363086609069243
2.0351040869207804
2.0363837378034857
2.008505554865039
2.028375828173052
2.0385613172277046
2.0305419075709805
2.037353448237042
2.0421565281146505
2.018659681890576
2.074319216880061
2.0326523714414213
1.9929925666503898
2.0449776947704996
2.0400115265097485
2.0318913586900917
2.0514753

1.98041582770132
1.9833521824136802
1.9975871634044637
2.021264922533847
1.985297002701719
2.007273175273191
2.0098952852271
1.9735641202833576
2.0195320297157537
2.0027002237581875
1.9809211001644718
2.010933673682739
2.0070869626802925
1.9802088925315404
2.014822806966147
1.9887398147466857
1.9714689190032746
2.0198716673934283
1.9933929868510927
1.9949770107517713
2.0050097571213814
1.9641096123225295
1.9891142688436088
2.0240930391028664
1.9760185070744969
1.96720819185333
2.012888694679124
1.9915256541797512
1.9866798973523023
1.999343776941168
1.9775187337293034
1.9654198839884796
1.9947774647984533
2.014584697954913
1.957613388095044
1.9955051877480463
2.0057247518944332
2.000395456761564
2.006065816567531
1.9939670855608425
1.9730049868907622
2.0136050719287644
1.987483622600736
1.9783375032310722
2.024621630759964
1.976880821064355
1.9490875449143954
2.018933227564232
1.9969890276833746
1.9809245232362127
2.0063600627309404
1.9986427550710872
1.9544399603705804
1.9851558820843

1.9367189559087103
1.9755024604379665
1.9601230824749312
1.9447455270836866
1.9862852750730076
1.9512449719917377
1.9708593055151884
1.960621376561934
1.9664248148570913
1.9382940897618617
1.9619604143841773
1.9624052543156583
1.9526838765588592
1.9592243267423783
1.9706495085216227
1.933141724774022
1.982918974718911
1.9700395635711083
1.913572512565626
1.9770689315173515
1.9528114984941485
1.943606022821708
1.9491206755633703
1.956581258932895
1.936484249729201
1.9855023761867656
1.9550856659563374
1.934727636608167
1.9644242260277758
1.962055009172685
1.939904587546677
1.9551128696261826
1.9649577950913686
1.9671701585019223
1.9777630174796055
1.9590791004821253
1.9645826651517675
1.9808127923524743
1.9491656813757323
1.945261809420291
1.9650955577628055
1.9629401454554336
1.9501046403741542
1.9349480567875414
1.9565309979199204
1.938681168419974
1.9620048603880458
1.9557637833266077
1.9638279042275824
1.965443900387136
1.9375197465046916
1.9344885611729095
1.9723655025921085
1.9520

1.924867203082871
1.937386683181845
1.9438138510859622
1.9297491361685652
1.9579504706441837
1.9299348128253448
1.9126918362045628
1.921022693178507
1.9196309288346274
1.9406038918908206
1.9272251284915525
1.931414842931315
1.916404596822929
1.9316321626834065
1.939711190198661
1.908602223507313
1.9386992958324156
1.9234019770260795
1.9199712701007436
1.9508401066239132
1.92416912122486
1.92313304002749
1.9328738122295004
1.9189799937912344
1.9132939545137675
1.9357087184503268
1.9229178002914962
1.9063597375739838
1.9289333340211177
1.9316947793267458
1.9198539113912223
1.9399089040937452
1.9364095802418464
1.907674194061606
1.942909386396383
1.9141577539130161
1.9177277084384508
1.9402907990273346
1.935662971684289
1.913809490180075
1.924685343899996
1.9226147591176372
1.912848648134076
1.942276606266557
1.9404679927171349
1.915427686889172
1.9223128840525616
1.9306042665779912
1.9030455693280013
1.927461584574969
1.930055292091457
1.9078153683371997
1.9281369134054134
1.931553077149

In [51]:
print(theta_stimato)
print(Th[theta_vero].vector)
print(best_answer())
print(best_arm())
print(W)

[-0.0649229   0.45792868  0.52047002]
[0.1, 0.4, 0.5]
1
1
[0.08058132568818505, 0.8528441041475785, 0.06657457016423643]


[1.9423824444604474, 3.175981218016338]

In [1595]:
b.pull_arm()
c.pull_arm()
b.pull_arm()

4.0553451251167205

In [1559]:
Y

[2.359809218555277,
 4.131155314683859,
 2.904517191004988,
 24.914421812754583,
 3.7649844287528778]

In [1560]:
N

[1, 3, 1]

In [1561]:
W = [1.1, 2.6, 1.3]

In [1562]:
arm_to_play()

2

In [1563]:
t = 4
stima_theta()

array([ 2.77437357, -0.06446984, -0.09043354])

In [1564]:
for i in range (0, num_arms):
    print(best_response(i))

1
0
0


In [1565]:
best_response(0)

1

In [1566]:
np.array(theta_stimato) - np.array(Th[best_response(0)].vector)

array([ 0.6, -0.1,  0.3])

In [1567]:
np.array(A[0].arm_vector)

array([1, 2, 3])

In [1568]:
p1 = np.matmul((np.array(theta_stimato) - np.array(Th[best_response(0)].vector)), np.array(A[0].arm_vector))
p1

1.2999999999999998

In [1569]:
math.sqrt(2*beta(t, 1/t**3))

3.8072546006004324

In [1570]:
np.sqrt(2*beta(t, 1/t**3))

3.9841738037605214

In [1571]:
k = 1000
time = k
delta = 1/k**3


In [1572]:
n = np.abs(np.random.normal(0, 1))
n

1.3485216370993358

In [1573]:
math.sqrt(math.log10(1/delta))

3.0

In [1574]:
d/2*math.log10(1 + (time*(l**2))/(n*d))

6.589557593201865

In [1575]:
math.sqrt(n/2)*m

8.211338615290858

In [1576]:
beta(k, 1/k**3)

7.45007683426267

In [1577]:
p2 = np.sqrt(2*beta(t, 1/t**3))*norm_2_mat(np.array(A[0].arm_vector), design_matrix(N) + np.random.normal(0, 1)*np.identity(d))

In [1578]:
calcola_U(0, 2)

269601.97156973823

In [1579]:
calcola_U(1, 2)

649542.4283968954

In [1580]:
calcola_U(2, 2)

32689597.500705175

In [1581]:
for i in range (0 , T//3 - t):
    a.pull_arm()
    b.pull_arm()
    c.pull_arm()
    t += 3

In [1582]:

stima_theta()

array([0.40123031, 0.59863247, 0.20280594])

In [1583]:
for i in range (0, num_arms):
    print(best_response(i))

1
0
0


In [1584]:
for i in range (0, num_arms):
    print(np.matmul(A[i].arm_vector, Th[theta_vero].vector))

2.2
3.6
24.0


In [1585]:
for i in range (0, num_arms):
    print(calcola_U(i, best_response(i))/100000000)

63.064612560315304
462.9371160274454
16389.492404270248
